# Import Library

Berikut merupakan library yang diperlukan

In [1]:
import pandas as pd
import pickle
import numpy as np
import streamlit as st
from sklearn.metrics.pairwise import cosine_similarity
import cv2

import warnings
warnings.filterwarnings("ignore")

# Test Program


Disini akan dilakukan test terhadap program Face Recognition yang telah dibuat. Nilai akurasi dan f1-score akan menjadi acuan dalam menilai performa program

In [2]:
# Load database yang berisi face embedding dan nama yang sudah teregistrasi
with open(r'app\database.pkl', 'rb') as file:
        database = pickle.load(file)

In [3]:
# Membuat list kosong untuk memuat face embedding dan nama yang sudah terigistrasi
registered_name=[]
registered_embedding=[]

# data dari data base dimasukkan ke dalam list dengan menggunakan for loop
for data in database:
            
            # Ambil embedding wajah dan nama dari data
            stored_embedding, stored_name = data
            
            # masukan ke list dengan .append
            registered_embedding.append(stored_embedding)
            registered_name.append(stored_name)

In [4]:
# membuat dataframe dengan list yang telah dibuat

df = pd.DataFrame({"Embedding": registered_embedding,'Name':registered_name })

# munculkan dataframe
df

,Embedding,Name
0,[[[[0.3254902 0.20000002 0.04313726 0.0509804...,Ali Asgar
1,[[[[0.6156863 0.43921572 0.32941177 0.2980392...,Artiwara Toon Kongmalai
2,[[[[0.9568628 0.9568628 0.9450981 0.9529412...,Boim Lebon
3,[[[[0.12941177 0.19607845 0.20784315 0.1647058...,Chen Sicheng
4,[[[[0.92549026 0.92549026 0.9294118 0.9294118...,Coki Pardede
5,[[[[0.41176474 0.42352945 0.45882356 0.4941176...,Eddy Katuari
6,[[[[0.69803923 0.7019608 0.70980394 0.7019608...,Fan Wei
7,[[[[0.18823531 0.1764706 0.09803922 0.0627451...,Kapil Sharma
8,[[[[0.13333334 0.07450981 0.2901961 0.3254902...,Kiku Sharda
9,[[[[0.882353 0.6901961 0.6431373 0.3098039...,Ma Dehua


In [5]:
Name_predict = []

In [6]:
def recognize_face2(embedding, database):
    # Inisialisasi nama dan nilai similarity terkecil
    name = 'Your face is not registered'
    max_similarity = 0.0
    
    # Loop setiap data di database
    for data in database:
        # Ambil embedding wajah dan nama dari data
        stored_embedding, stored_name = data
        
        # Hitung similarity antara embedding wajah terdeteksi dengan embedding wajah di database
        similarity = cosine_similarity(embedding.reshape(1, -1), stored_embedding.reshape(1, -1))[0][0]
        
        # Jika similarity lebih besar dari nilai terbesar sebelumnya, update nama dan nilai terbesar
        if similarity > max_similarity:
            max_similarity = similarity
            name = stored_name
    
    # Kembalikan nama yang terdeteksi
    return name

In [7]:
# Import library yang dibutuhkan
import cv2
import numpy as np
import pickle
from scipy.spatial.distance import euclidean

# Fungsi untuk mendeteksi wajah dan mengembalikan embedding wajah
def detect_and_embed(image):
    # Mendeteksi wajah dengan OpenCV
    faces = cv2.CascadeClassifier(r'app\haarcascade_frontalface_default.xml').detectMultiScale(image, 1.3, 5)
    
    # Jika tidak ada wajah yang terdeteksi, kembalikan None
    if len(faces) == 0:
        return None
    # Jika terdeteksi lebih dari satu wajah, kembalikan "more_than_1"
    elif len(faces) > 1:
        return 'more_than_1'
    # Jika terdeteksi satu wajah, kembalikan embedding wajah
    else:
        x, y, w, h = faces[0]
        face = image[y:y+h, x:x+w]
        face = cv2.resize(face, (160, 160))
        embedding = cv2.dnn.blobFromImage(face, 1.0/255, (96, 96), (0, 0, 0), swapRB=True, crop=True)
        return embedding

# Fungsi untuk mengecek kesamaan wajah dengan wajah di database
def recognize_face(embedding, database):
    # Inisialisasi nama dan nilai similarity terkecil
    name = 'Your face is not registered'
    min_similarity = 20.0 
    
    # Loop setiap data di database
    for data in database:
        # Ambil embedding wajah dan nama dari data
        stored_embedding, stored_name = data
        
        # Hitung similarity antara embedding wajah terdeteksi dengan embedding wajah di database
        similarity = np.linalg.norm(embedding - stored_embedding)
        
        # Jika similarity lebih kecil dari nilai terkecil sebelumnya, update nama dan nilai terkecil
        if similarity < min_similarity:
            min_similarity = similarity
            name = stored_name
    
    # Kembalikan nama yang terdeteksi
    return name

# Baca database wajah dari file pickle
with open(r'app\database.pkl', 'rb') as file:
    database = pickle.load(file)


for i in df.Name:
    # Baca gambar wajah
    image = cv2.imread(rf'foto\test\Ali Asgar.jpg', cv2.IMREAD_GRAYSCALE)

    # Mendeteksi wajah dan mengembalikan embedding wajah
    embedding = detect_and_embed(image)

    # Jika tidak ada wajah yang terdeteksi, tampilkan pesan error
    if embedding is None:
        print('No face detected')
    # Jika terdeteksi lebih dari satu wajah, tampilkan pesan error
    elif embedding == "more_than_1":
        print('More than one face is detected')
    # Jika terdeteksi satu wajah, lakukan pengenalan wajah
    else:
        # Lakukan pengenalan wajah dengan database
        name = recognize_face2(embedding, database)
        
        # Tambahkan nama ke dalam list Name_predict
        print(f'Name: {name}')


Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
Name: Wang Baoqiang
